## Install necessary libraries

In [ ]:
!pip install sentence_transformers -q
!pip install pinecone-client -q

## Load data

In [ ]:
cricket_news = """
The T20 World Cup 2024 is in full swing, bringing excitement and drama to cricket fans worldwide.
India's team, captained by Rohit Sharma, is preparing for a crucial match against Ireland, with standout player Jasprit Bumrah expected to play a pivotal role in their campaign.
The tournament has already seen controversy, particularly concerning the pitch conditions at Nassau County International Cricket Stadium in New York, which came under fire after a low-scoring game between Sri Lanka and South Africa.
"""

football_news = """
The world of football is buzzing with excitement as major tournaments and league matches continue to captivate fans globally.
In the UEFA Champions League, the semi-final matchups have been set, with defending champions Real Madrid set to face Manchester City, while Bayern Munich will take on Paris Saint-Germain.
Both ties promise thrilling encounters, featuring some of the best talents in world football.
"""

election_news = """
As election season heats up, the latest developments reveal a highly competitive atmosphere across several key races.
The presidential election has seen intense campaigning from all major candidates, with recent polls indicating a tight race.
Incumbent President Jane Doe is seeking re-election on a platform of economic stability and healthcare reform, while her main rival, Senator John Smith, focuses on education and climate change initiatives."""


ai_revolution_news = """
The AI revolution continues to transform industries and reshape the global economy.
Significant advancements in artificial intelligence have led to breakthroughs in healthcare, with AI-driven diagnostics improving patient outcomes and reducing costs.
Autonomous systems are becoming increasingly prevalent in logistics and transportation, enhancing efficiency and safety."""

## Perform embeddings on data

In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-mpnet-base-v2")

In [ ]:
embeddings = embedding_model.encode([cricket_news, football_news, election_news, ai_revolution_news])

In [ ]:
embeddings

In [ ]:
len(embeddings[0])

## Initiate Pinecone

In [ ]:
from pinecone import Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key="")
spec = ServerlessSpec(cloud='aws', region='us-east-1')

## Create Index

In [ ]:
pc.create_index("example-index", dimension=768, metric="cosine", spec=spec)

In [ ]:
pc.list_indexes()

## Use Index

In [ ]:
index = pc.Index("example-index")

In [ ]:
index.describe_index_stats()

## Add data to Pinecone Index

In [ ]:
index.upsert([
    {"id":"id1", "values":embeddings[0], "metadata":{'source': 'cricket'}},
    {"id":"id2", "values":embeddings[1], "metadata":{'source': 'football'}},
    {"id":"id3", "values":embeddings[2], "metadata":{'source': 'election'}},
    {"id":"id4", "values":embeddings[3], "metadata":{'source': 'ai_revolution'}}
])

In [ ]:
index.describe_index_stats()

## Similarity Search

In [ ]:
query = "technology"
query_embedding = embedding_model.encode(query).tolist()

In [ ]:
len(query_embedding)

In [ ]:
similar_docs = index.query(vector=query_embedding, top_k=2, include_metadata=True)
similar_docs

## CRUD operations on Vector Database

#### Add data

In [ ]:
blockchain_news = """
The blockchain industry continues to evolve rapidly, marked by significant technological advancements and regulatory developments.
This month, the spotlight is on the launch of Ethereum 3.0, which promises enhanced scalability and security features.
This upgrade is expected to drastically reduce transaction fees and increase processing speeds, making decentralized applications (dApps) more efficient and user-friendly.
"""

In [ ]:
embedding_query = embedding_model.encode(blockchain_news).tolist()

In [ ]:
index.describe_index_stats()

In [ ]:
index.upsert([{"id":"id5", "values":embedding_query, "metadata":{"source":"blockchain"}}])

In [ ]:
index.describe_index_stats()

In [ ]:
query_embedding = embedding_model.encode("technology").tolist()
similar_docs = index.query(vector=query_embedding, top_k=2, include_metadata=True)
similar_docs

#### Read data

In [ ]:
results = index.fetch(ids=['id1', 'id3'])

In [ ]:
for k in results["vectors"]:
  print(results["vectors"][k]['metadata'])

#### Update data

In [ ]:
embedding_query = embedding_model.encode("This is sample document about generative AI").tolist()
index.upsert([("id3", embedding_query, {"source":"gen ai"})])

In [ ]:
index.fetch(ids=['id3'])

#### Delete data

In [ ]:
index.fetch(ids=['id2'])

In [ ]:
index.delete(ids=['id2'])

In [ ]:
index.fetch(ids=['id2'])

## Delete Pinecone Index

In [ ]:
pc.delete_index('example-index')